In [66]:

import pickle
import torch
import sys
import torch
from torch import nn

# Setup

### Data formatting

In [67]:
import json
import os

class Item:
    def __init__(self, key, value, type:str):
        self.key = key
        self.type = type
        if type.startswith("Tensor"):
            self.value = {"size":value.size(),"values":value.flatten().tolist()}
        else:
            self.value = value

    def to_dict(self):
        return {self.key: {self.type: self.value}}


def to_file(name:str,items:list):
    path = "test-files/"+name+".json"
    values = {}
    for item in items:
        values.update(item.to_dict())
    output = {"values": values}
    
    data = json.dumps(output, indent=4)
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, "w") as f:
        f.write(data)

def random_tensor(shape:list):
    length = 1 
    for dim in shape:
        length*=dim

    values = []
    value=1.0
    for i in range(length):
        value+=1
        values.append(value)
    return torch.tensor(values).view(shape)


## Common
#### LayerNorm2d

In [68]:
from segment_anything.modeling.common import LayerNorm2d

layer_norm = LayerNorm2d(256,0.1)
items = [Item("weight", layer_norm.weight, "TensorFloat"), Item("bias", layer_norm.bias, "TensorFloat"), Item("eps", layer_norm.eps, "Float")]
to_file("layer_norm_2d",items)

# Forward
input = random_tensor([2,256,16,16])
output = layer_norm(input)
items = [Item("input", input, "TensorFloat"), Item("output", output, "TensorFloat")]
to_file("layer_norm_2d_forward",items)

#### MLPBlock

In [69]:
from segment_anything.modeling.common import MLPBlock

mlp_block = MLPBlock(256,256,nn.GELU)
mlp_block.lin1.weight.data = random_tensor([256,256])
mlp_block.lin2.weight.data = random_tensor([256,256])
mlp_block.lin1.bias.data = random_tensor([256])
mlp_block.lin2.bias.data = random_tensor([256])
items=[Item("lin1", mlp_block.lin1.weight, "TensorFloat"), Item("lin2", mlp_block.lin2.weight, "TensorFloat")]
to_file("mlp_block",items)

# Forward
input = random_tensor([256,256])
output = mlp_block(input)
items = [Item("input", input, "TensorFloat"), Item("output", output, "TensorFloat")]
to_file("mlp_block_forward",items)

#### Activation

In [70]:
# Gelu
gelu = nn.GELU()
input = random_tensor([256,256])
output = gelu(input)
items = [Item("input", input, "TensorFloat"), Item("output", output, "TensorFloat")]
to_file("activation_gelu",items)

# ReLU
relu = nn.ReLU()
input = random_tensor([256,256])
output = relu(input)
items = [Item("input", input, "TensorFloat"), Item("output", output, "TensorFloat")]
to_file("activation_relu",items)
